# Packages 

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import logging
base_dir = '../../'
sys.path.append(base_dir)
import os
from utils import *

import os
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from gensim.similarities.annoy import AnnoyIndexer

from tqdm import tqdm


from annoy import AnnoyIndex
import polars as pl
import implicit
import scipy.sparse as sps
from utils import str2list
from src.config import raw_data_session_id_dir, candidate_file_name, raw_data_session_id_dir
from src.case_analysis import show_single_case
from lightgbm import LGBMRanker
from utils import *


/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


SyntaxError: invalid syntax (case_analysis.py, line 16)

# Config

In [59]:
debug = False


if debug:
    n_rows = 1000
else:
    n_rows = None


# Read data 

In [60]:
! ls {base_dir}/{raw_data_session_id_dir}

product_unique2id.json		    sessions_test_task2.parquet
products_train.parquet		    sessions_test_task2_phase1.parquet
sessions_eval.parquet		    sessions_test_task3.parquet
sessions_test_task1.parquet	    sessions_test_task3_phase1.parquet
sessions_test_task1_phase1.parquet  sessions_train.parquet


In [61]:
test_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task3.parquet'), n_rows=n_rows).with_columns(pl.col('prev_items').apply(str2list))
product_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 
                                          'products_train.parquet'))


In [62]:
test_pl.collect().describe()

describe,prev_items,locale,session_id
str,str,str,f64
"""count""","""56427""","""56427""",56427.0
"""null_count""","""0""","""5""",0.0
"""mean""",null,null,4.39421e6
"""std""",null,null,16289.216157
"""min""",null,"""DE""",4.365997e6
"""max""",null,"""UK""",4.422423e6
"""median""",null,null,4.39421e6


In [67]:
test_pl.select('locale').collect().to_series().value_counts()

locale,counts
str,u32
"""DE""",10000
"""JP""",10000
"""ES""",6422
null,5
"""UK""",10000
"""IT""",10000
"""FR""",10000


In [69]:
test_pl.filter(pl.col('locale')=='UK').collect().sample(3)

prev_items,locale,session_id
list[str],str,i64
"[""B09HSJX8XJ"", ""B09KLD82B7"", ""B09KKYH7N2""]","""UK""",4422066
"[""B0BJKNDR1K"", ""B07P177RK8"", … ""B095H7C8TX""]","""UK""",4422246
"[""B0B9HMTVBK"", ""B075KDGGWM"", … ""B00DEB5GQU""]","""UK""",4413201


In [71]:
# test_pl.tail(5).collect()

In [72]:
# product_pl.head().collect()

# Case study 

In [74]:
session_id = 4365999

session_pl = test_pl
product_pl = product_pl



In [73]:
show_single_case(session_id=4422246, session_pl=session_pl, product_pl=product_pl)

++++++++++++++++++++
0
++++++++++++++++++++
===title====
    camouflage net 54pcs Magnetic Balls and Rods Building Sticks Blocks Set DIY Assembly Stacking STEM Magnet Toys Educative Building Rod Set for 3+ Years Old Kids

===desc====
    【54pcs】The magnetic building block bat set includes 54 building blocks, including 8* smile ball, 8* curved bar and 38* magnetic bar. A sufficient number can build various 2D models and 3D buildings.

unique_id                             960085
price                                  24.99
brand                         camouflage net
color                                   None
size                                    None
model                                   None
material     Acrylonitrile Butadiene Styrene
author                                  None
Name: 0, dtype: object


++++++++++++++++++++
1
++++++++++++++++++++
===title====
    Geomag 350 Confetti Confetti-32pcs, Mixed, 32 Pieces

===desc====
    Explore your imagination: Geomag magnetic kits

In [33]:
session_details['title'].to_pandas().values

array(['Correa Xiaomi Mi Band 4, Superpow 3 Piezas Correas para Xiaomi Mi Band 3, Pulsera xiaomi mi band 4 / Xiaomi band 3 Correa de Repuesto Suave, Transpirable, Resistente al Sudor (negro/gris/azul marino)',
       'Correa Xiaomi Mi Band 4, Superpow 3 Piezas Correas para Xiaomi Mi Band 3, Pulsera xiaomi mi band 4 / Xiaomi band 3 Correa de Repuesto Suave, Transpirable, Resistente al Sudor (gris/rosa/amarillo)'],
      dtype=object)

In [39]:
session_details.to_pandas().columns

Index(['prev_items', 'locale', 'session_id', 'title', 'price', 'brand',
       'color', 'size', 'model', 'material', 'author', 'desc', 'unique_id'],
      dtype='object')

++++++++++++++++++++
0
++++++++++++++++++++
===title====
    Correa Xiaomi Mi Band 4, Superpow 3 Piezas Correas para Xiaomi Mi Band 3, Pulsera xiaomi mi band 4 / Xiaomi band 3 Correa de Repuesto Suave, Transpirable, Resistente al Sudor (negro/gris/azul marino)

===desc====
    【3 colores para elegir】 :Pulsera Xiaomi Mi Band 4 proporcionamos 3 colores diferentes, puede cambiar el color de acuerdo con su estado de ánimo y preferencia o combinarlo con su ropa todos los días

unique_id                       1423913
price                              6.99
brand                          superpow
color        negro / gris / azul marino
size                               None
model                              None
material                           None
author                             None
Name: 0, dtype: object


++++++++++++++++++++
1
++++++++++++++++++++
===title====
    Correa Xiaomi Mi Band 4, Superpow 3 Piezas Correas para Xiaomi Mi Band 3, Pulsera xiaomi mi band 4 / Xiaomi band 3 Co